## Missing Fingers

In [1]:
import warnings
from typing import List

import pyro
import pyro.distributions as dist
import torch

import probability_on_trial as pot

Building Core model:

In [40]:
pyro.set_rng_seed(42)

import matplotlib.pyplot as plt 


pr_identity = 1/10e6 # Prior probability that the suspect is guilty if they are say from Sweden  
pr_identity_if_isis = 1/ 300 # say we estimate total ISIS force at that time to be 100k
rmp_fingers_sweden = 2 / 20000 # RMP for Sweden
pr_fingers_match_if_isis=2 / 5000 # RMP for ISIS memebers; for now a precise number4
pr_isis = 300/10e6 # Prior probability that the suspect is from ISIS


def swedish_fingers_guilt_precise_core(
    pr_identity: float = pr_identity, 
    pr_identity_if_isis: float =pr_identity_if_isis,     
    rmp_fingers_sweden: float =rmp_fingers_sweden,  
    pr_fingers_match_if_isis: float = pr_fingers_match_if_isis,  
    pr_isis = pr_isis,
    num_particles=3e7,  
):

    with pyro.plate("particles", size=num_particles):

        # not important, here, important for the probability of evidential scenarios
        isis = pyro.sample(
            "isis", dist.Bernoulli(pr_isis)
        )

        guilty_sweden = pyro.sample("guilty_sweden", dist.Bernoulli(pr_identity))

        guilty_isis = pyro.sample("guilty", dist.Bernoulli(
            (1-isis) * pr_identity + isis * pr_identity_if_isis))
    

        fingers_match_sweden = pyro.sample(
            "fingers_match_not_isis",
            # if guilty then 1
            # if not guilty then sample with probability rmp
            dist.Bernoulli(guilty_sweden * 1 + (1 - guilty_sweden) * (rmp_fingers_sweden)),
        )


        # this is with isis as an informative node, 
        # but will only give the right result if the model is conditoned on isis
        # which we meed to do anyway
        # for MC estimation, as being in isis is rare
        fingers_match_isis =   pyro.sample("fingers_match_isis",
            dist.Bernoulli(
                guilty_isis * 1
                + (1 - guilty_isis) * pr_fingers_match_if_isis
                                ),
        )
        

    # Evidence mask 1: fingers match
    evidence_mask_fingers_sweden = (fingers_match_sweden).bool()
    guilty_masked_fingers_sweden = pyro.deterministic(
        "guilty_masked_fingers_sweden",
        torch.where(evidence_mask_fingers_sweden, guilty_sweden, torch.tensor(float("nan"))),
    )

    posterior_guilty_fingers_sweden = pyro.deterministic(
        "posterior_guilty_fingers_sweden", torch.nanmean(guilty_masked_fingers_sweden, axis=0)
    )

    # Mask 2: fingers match, isis
    evidence_mask_fingers_isis = (fingers_match_isis).bool()
    

    guilty_masked_fingers_isis = pyro.deterministic(
        "guilty_masked_fingers_isis",
        torch.where(evidence_mask_fingers_isis, guilty_isis, torch.tensor(float("nan"))),
    )

    posterior_guilty_fingers_isis = pyro.deterministic(
        "posterior_guilty_fingers_isis"        ,
        torch.nanmean(guilty_masked_fingers_isis, axis=0),
    )

    return {
        "posterior_guilty_fingers_sweden": posterior_guilty_fingers_sweden,
        "posterior_guilty_fingers_isis": posterior_guilty_fingers_isis,
    }



with pyro.condition(data={"isis": torch.tensor(1.0)}):
    posterior_isis = swedish_fingers_guilt_precise_core()['posterior_guilty_fingers_isis']


def swedish_fingers_guilt_precise_estimator(**kwargs):

    posterior_sweden = swedish_fingers_guilt_precise_core(**kwargs)['posterior_guilty_fingers_sweden']

    with pyro.condition(data={"isis": torch.tensor(1.0)}):
        posterior_isis = swedish_fingers_guilt_precise_core(**kwargs)['posterior_guilty_fingers_isis']

    return posterior_sweden, posterior_isis



pyro.set_rng_seed(422323)

posterior_sweden, posterior_isis = swedish_fingers_guilt_precise_estimator()

print(posterior_sweden, posterior_isis)

tensor([0.0033])
fingers_match_sweden tensor([1.0000e-04, 1.0000e+00])
fingers_match_isis tensor([4.0000e-04, 1.0000e+00])
tensor([1.0000e-07, 3.3333e-03])
fingers_match_sweden tensor([1.0000e-04, 1.0000e+00])
fingers_match_isis tensor([4.0000e-04, 1.0000e+00])
tensor([0.0033])
fingers_match_sweden tensor([1.0000e-04, 1.0000e+00])
fingers_match_isis tensor([4.0000e-04, 1.0000e+00])
tensor(0.0007) tensor(0.8952)


In [5]:
# requires pyro 1.9.1
#pyro.render_model(swedish_fingers_guilt_model, render_deterministic=True)

In [43]:
posterior_guilt_sweden = torch.tensor(
    [
        1 - posterior_sweden,
        posterior_sweden,]
)

posterior_guilt_isis = torch.tensor(
    [
    1 - posterior_isis,
        posterior_isis
    ]
)

prior_guilt = torch.tensor([1 - pr_identity, pr_identity])
prior_guilt_isis = torch.tensor([1 - pr_identity_if_isis, pr_identity_if_isis])

print("posterior_guilt_no_isis_info", posterior_guilt_sweden)
print("posterior_guilt_isis", posterior_guilt_isis)

# need to rethink this
probs_of_evidece =  torch.tensor(
        [
        1- (pr_isis * pr_fingers_match_if_isis), pr_isis * pr_fingers_match_if_isis
        ]
    )
# probs_of_evidence = torch.tensor(
#     [pr_fingers_match_if_not_isis, pr_fingers_match_if_isis]
# )
# print(probs_of_evidence)

posterior_guilt_no_isis_info tensor([9.9932e-01, 6.8213e-04])
posterior_guilt_isis tensor([0.1048, 0.8952])


In [54]:
expected_weights = expected_weight(
    probs_of_evidence,
    prior_guilt,
    [posterior_guilt_not_isis_mean, posterior_guilt_isis_mean],
)

print("Expected Weights:", expected_weights)

tensor([0.9179, 0.9160])
Expected Weights: {'expected_weight': tensor(-1.4350e-06), 'weight_prior': 0.9192068800330162, 'posterior_weights': tensor([0.9179, 0.9160]), 'weight_changes': tensor([-0.0013, -0.0033]), 'weighted_weight_changes': tensor([-1.3228e-07, -1.3027e-06])}
